# 11 | 省下钱买显卡，如何利用开源模型节约成本？

## 在 Colab 线上使用免费 GPU


## HuggingfaceEmbedding，你的开源伙伴

安装依赖
```
conda install -c conda-forge sentence-transformers
```


In [9]:

import openai, os
import faiss
from llama_index import SimpleDirectoryReader, LangchainEmbedding, ServiceContext, GPTVectorStoreIndex, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from llama_index.node_parser import SimpleNodeParser

openai.api_key = ""

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('../geektime-ai-course/data/faq/').load_data()
nodes = parser.get_nodes_from_documents(documents)

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))
service_context = ServiceContext.from_defaults(embed_model=embed_model)

dimension = 768
faiss_index = faiss.IndexFlatIP(dimension)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)


Created a chunk of size 130, which is longer than the specified 100


In [21]:

openai.api_key = os.environ.get("OPENAI_API_KEY")

query_engine = index.as_query_engine(verbose=True)
response = query_engine.query("请问你们海南能发货吗？")
print(response)


A: 是的，我们支持海南地区的发货。您可以在下单时选择收货地址，我们会根据您的地址信息提供相应的物流服务。


In [22]:

response = query_engine.query("你们用哪些快递公司送货？")
print(response)


我们与顺丰速运、圆通速递、申通快递、韵达快递、中通快递、百世快递等多家知名快递公司合作。


In [23]:

response = query_engine.query("你们的退货政策是怎么样的？")
print(response)


A: 自收到商品之日起7天内，如产品未使用、包装完好，您可以申请退货。某些特殊商品可能不支持退货，请在购买前查看商品详情页面的退货政策。


由于本节后面的内容无法在 MacMini M1 上面调通，故本节 11-02.ipynb 全部在 colab 上面运行的，Runtime Type 请选择 GPU